In [ ]:
import pandas as pd
import numpy as np
import pickle

import plotly.express as px
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import plotly.graph_objects as go

In [ ]:
with open('../data/nash_hosp_referrers_b.pickle', 'rb') as file:
    nash_hosp_referrers_b = pickle.load(file)

In [ ]:
nash_hosp_referrers_b['hospital'] = (
    nash_hosp_referrers_b['hospital']
    .str.replace('SETON CORPORATION', 'SAINT THOMAS')
    .str.replace('SAINT THOMAS WEST HOSPITAL', 'SAINT THOMAS')
    .str.replace('HCA HEALTH SERVICES OF TENNESSEE, INC.', 'HCA/TRISTAR')
    .str.replace('HTI MEMORIAL HOSPITAL CORPORATION', 'HCA/TRISTAR')
    .str.replace('NASHVILLE GENERAL HOSPITAL', 'NASHVILLE GENERAL')
    .str.replace('VANDERBILT UNIVERSITY MEDICAL CENTER', 'VUMC')
)

#add a column that calculates the total wait in days for each providers' referrals
nash_hosp_referrers_b['total_wait'] = (nash_hosp_referrers_b['transaction_count'] 
                                       * 
                                       nash_hosp_referrers_b['average_day_wait'])

In [ ]:
# Create interactive table that allows you to choose a hospital and view info about referrals by specialty
@interact(Hospital = np.sort(nash_hosp_referrers_b['hospital'].unique()))
def interactive_plot(Hospital):
    
    df = (nash_hosp_referrers_b
          .loc[nash_hosp_referrers_b['hospital'] == Hospital]
          .groupby('sub_specialty')
          .agg(total_providers = ('from_npi', 'count'),
               total_transactions = ('transaction_count', 'sum'),
               total_patients = ('patient_count', 'sum'),
               total_states = ('location_address_state_name', 'nunique'),
               total_wait = ('total_wait', 'sum')
               
          )
          .reset_index()
          .sort_values('total_transactions', ascending = False))
    
    df['avg_day_wait'] = round(df['total_wait'] / df['total_transactions'], 2)
    
    fig = go.Figure(data=[go.Table(
        header=dict(values= ['Referring Sub-Specialty',
                             'Total Referring Providers',
                             'Total Referral Transactions',
                             'Total Patients Referred',
                             'Average Wait (days)',
                             'Total Referring States'],
                    fill_color='#00618c',
                    font_color= '#ffffff',
                    align='left'),
        cells=dict(values=[df.sub_specialty, 
                           df.total_providers,
                           df.total_transactions,
                           df.total_patients,
                           df.avg_day_wait,
                           df.total_states],
                   fill_color='#daf7ff',
                   align='left'))
    ])

    fig.write_html("../data/visuals/specialty_table.html")
    
    fig.show()